### Code to create monthly sum,avg

In [1]:
%env CURL_CA_BUNDLE=/etc/ssl/certs/ca-certificates.crt

env: CURL_CA_BUNDLE=/etc/ssl/certs/ca-certificates.crt


In [2]:
import sys
import datetime
import os
import time
import rasterio
import numpy as np
from glob import glob
import os

In [3]:

def create_s3_list_of_days(main_prefix, year, output_name='etasw_'):
    the_list = []
    for i in range(1,366):
        month = f'{i:03d}'
        file_object = main_prefix + '/' + str(year) + '/' + output_name + str(year) + month + '.tif'
        the_list.append(file_object)
    return the_list


In [4]:
main_prefix = 's3://dev-et-data/in/DelawareRiverBasin/PPT'
year = 2003
output_name = 'PPT_'
ppt_list = create_s3_list_of_days(main_prefix, year, output_name)
#ppt_list 



In [5]:
#read in file with rasterio
def read_file(file):
    with rasterio.open(file) as src:
        return(src.read(1))

In [6]:
def monthly_average(file_list, out_dir):
# what months to summarize
    start_mon = 1 #start month
    end_mon = 12 #end month


    #loop through month 1,2,..12    
    for i in range(start_mon,(end_mon+1)): 
        print('Month averaged up is: ' + str(i))
        Listras = [] 
        for et_in in file_list:
            doy = int(et_in.split('.')[0][-3:])
            #doy = int(et_in[-3:])
            #print 'Day of the year: ' + str(doy)
            datea = str(datetime.date(year,1,1) + datetime.timedelta(doy-1))
            mon = int(datea.split('-')[1])
            #print 'Month is: ' + str(mon)
            if mon == i: #if month = i then append grid to list for summing up
                Listras.append(et_in)
        #print('daily grids for month ' + str(i) + ' :')
        #print(Listras)
        if Listras == []:
            print('No daily data for month' + str(i) + ' available..continue to next month')
            continue
        else:
            # Read all data as a list of numpy arrays 
            array_list = [read_file(x) for x in Listras]
            # Perform averaging
            array_out = np.nanmean(array_list, axis=0)

            # Get metadata from one of the input files
            with rasterio.open(file_list[0]) as src:
                meta = src.meta
            meta.update(dtype=rasterio.float32)

            # Write output file
            out_name = 'ppt_avg_' + str(year) + (('0'+ str(i))[-2:]) +'.tif'
            with rasterio.open(out_dir + '/' + out_name, 'w', **meta) as dst:
                dst.write(array_out.astype(rasterio.float32), 1)

            print('Created monthly grid!', out_name)


In [7]:
def _mkdir(directory):
    try:
        os.stat(directory)
    except:
        os.mkdir(directory)

In [8]:
_mkdir('./junkbox')

In [9]:

out_dir = './junkbox'
for year in range(2003,2016):
    file_list = create_s3_list_of_days(main_prefix, year, output_name)
    monthly_average(file_list, out_dir)

Month averaged up is: 1
Created monthly grid!
Month averaged up is: 2
Created monthly grid!
Month averaged up is: 3
Created monthly grid!
Month averaged up is: 4
Created monthly grid!
Month averaged up is: 5
Created monthly grid!
Month averaged up is: 6
Created monthly grid!
Month averaged up is: 7
Created monthly grid!
Month averaged up is: 8
Created monthly grid!
Month averaged up is: 9
Created monthly grid!
Month averaged up is: 10
Created monthly grid!
Month averaged up is: 11
Created monthly grid!
Month averaged up is: 12
Created monthly grid!
Month averaged up is: 1
Created monthly grid!
Month averaged up is: 2
Created monthly grid!
Month averaged up is: 3
Created monthly grid!
Month averaged up is: 4
Created monthly grid!
Month averaged up is: 5
Created monthly grid!
Month averaged up is: 6
Created monthly grid!
Month averaged up is: 7
Created monthly grid!
Month averaged up is: 8
Created monthly grid!
Month averaged up is: 9
Created monthly grid!
Month averaged up is: 10
Create

In [10]:
! ls -lh ./junkbox

ppt_avg_200301.tif  ppt_avg_200604.tif	ppt_avg_200907.tif  ppt_avg_201210.tif
ppt_avg_200302.tif  ppt_avg_200605.tif	ppt_avg_200908.tif  ppt_avg_201211.tif
ppt_avg_200303.tif  ppt_avg_200606.tif	ppt_avg_200909.tif  ppt_avg_201212.tif
ppt_avg_200304.tif  ppt_avg_200607.tif	ppt_avg_200910.tif  ppt_avg_201301.tif
ppt_avg_200305.tif  ppt_avg_200608.tif	ppt_avg_200911.tif  ppt_avg_201302.tif
ppt_avg_200306.tif  ppt_avg_200609.tif	ppt_avg_200912.tif  ppt_avg_201303.tif
ppt_avg_200307.tif  ppt_avg_200610.tif	ppt_avg_201001.tif  ppt_avg_201304.tif
ppt_avg_200308.tif  ppt_avg_200611.tif	ppt_avg_201002.tif  ppt_avg_201305.tif
ppt_avg_200309.tif  ppt_avg_200612.tif	ppt_avg_201003.tif  ppt_avg_201306.tif
ppt_avg_200310.tif  ppt_avg_200701.tif	ppt_avg_201004.tif  ppt_avg_201307.tif
ppt_avg_200311.tif  ppt_avg_200702.tif	ppt_avg_201005.tif  ppt_avg_201308.tif
ppt_avg_200312.tif  ppt_avg_200703.tif	ppt_avg_201006.tif  ppt_avg_201309.tif
ppt_avg_200401.tif  ppt_avg_200704.tif	ppt_avg_201007.tif  ppt_a